In [3]:
import psycopg2
from psycopg2 import sql
import time
import random
from collections import defaultdict
import concurrent.futures
import json
from pprint import pprint

In [4]:
data_collection = defaultdict(int)

# connection parameters

In [5]:
num_users = 1000
num_items = 1000
num_queries = 700
# max_workers = 32

In [6]:
host="ec2-35-163-19-216.us-west-2.compute.amazonaws.com"  # Replace with your EC2 instance's public DNS
port = "5432"
dbname = "mydatabase"
user = "danlandy"
password = "danlandy"
# file_name = 't2.micro.data.json'

# drop tables

In [7]:
def drop_all_tables():
    try:
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        print("Connected to the database successfully")

        # Create a cursor object
        cur = conn.cursor()

        # Define the SQL queries to drop the tables
        drop_queries = [
            "DROP TABLE IF EXISTS users_items;",
            "DROP TABLE IF EXISTS users;",
            "DROP TABLE IF EXISTS items;"
        ]
        
        # Execute each drop query
        for query in drop_queries:
            cur.execute(query)
        
        # Commit the changes
        conn.commit()
        print("Tables dropped successfully")

        # Close the cursor and connection
        cur.close()
        conn.close()
        print("Connection closed")

    except Exception as e:
        print(f"Unable to connect to the database or drop tables: {e}")


# measure the time for connecting the database and time for fetch the version data

In [8]:
def measure_connection_time(): 
    try:
        # Capture the start time
        start_time = time.time()

        # Connect to the PostgreSQL server
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )

        # Capture the end time
        end_time = time.time()

        # Calculate the connection duration
        connection_time = end_time - start_time
        print(f"Time taken to connect to the database: {connection_time:.6f} seconds")
        data_collection['connection_time'] = connection_time
        
        # Create a cursor object
        cur = conn.cursor()

        # Execute a SQL query
        start_time = time.time()
        cur.execute("SELECT version();")
        result = cur.fetchone()
        end_time = time.time()
        version_fetch_time = end_time - start_time
        
        
        print(f"Time taken to fetch the version data : {version_fetch_time:.6f} seconds")
        
        print("PostgreSQL version:", result)

        # Close the cursor and connection
        cur.close()
        conn.close()
        print("Connection closed")

    except Exception as e:
        print(f"Unable to connect to the database: {e}")


# create users table

In [9]:
def measure_creating_users_table():
    try:
        start_time_connection = time.time(); 
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        
        # Create a cursor object
        cur = conn.cursor()

        # Create the 'user_data' table
        create_table_query = '''
        CREATE TABLE users (
            user_id serial  PRIMARY KEY,
            first_name VARCHAR(50),
            last_name VARCHAR(50)
        );
        '''
        
        start_time = time.time()
        cur.execute(create_table_query)
        conn.commit()
        end_time = time.time()
        
        elapsed_time = end_time - start_time; 
        elapsed_time_2 = end_time - start_time_connection; 
        
        data_collection['create_user_table'] = elapsed_time_2
        # data_collection['create_user_table'].append(elapsed_time)
        
        print(f"Time taken to create users table and establishing connections : {elapsed_time_2:.6f} seconds")
        print(f"Time taken to create users table with established connections : {elapsed_time:.6f} seconds")
            
        print("Table 'users' created successfully")

        # Close the cursor and connection
        cur.close()
        conn.close()
        print("Connection closed")

    except Exception as e:
        print(f"Unable to connect to the database or create table: {e}")


# create Items table

In [10]:
def measure_create_items_table():
    try:
        start_time_connection = time.time() 
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        print("Connected to the database successfully")

        # Create a cursor object
        cur = conn.cursor()

        # Create the 'items' table within the 'items' schema
        create_table_query = '''
        CREATE TABLE items (
            item_id serial PRIMARY KEY,
            item_name VARCHAR(100),
            value1 varchar(100)
        );
        '''
        start_time = time.time()
        cur.execute(create_table_query)
        conn.commit()
        
        end_time = time.time()
        elapsed_time = end_time - start_time; 
        elapsed_time_2 = end_time - start_time_connection; 
        
        data_collection['create_item_table'] = elapsed_time_2
        # data_collection['create_item_table'].append(elapsed_time)
        
        print(f"Time taken to create items table and establishing connections : {elapsed_time_2:.6f} seconds")
        print(f"Time taken to create items table with established connections : {elapsed_time:.6f} seconds")
        
        print("Table 'items' created successfully")

        # Close the cursor and connection
        cur.close()
        conn.close()
        print("Connection closed")

    except Exception as e:
        print(f"Unable to connect to the database or create schema/table: {e}")


# create user-item table

In [11]:
def measure_create_users_items_table():
    try:
        start_time_connection = time.time() 
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        print("Connected to the database successfully")

        # Create a cursor object
        cur = conn.cursor()

        # Create the 'users_items' table
        create_table_query = '''
        CREATE TABLE users_items (
            user_id INTEGER NOT NULL,
            item_id INTEGER NOT NULL,
            PRIMARY KEY (user_id, item_id),
            FOREIGN KEY (user_id) REFERENCES users(user_id),
            FOREIGN KEY (item_id) REFERENCES items(item_id)
        );
        '''
        start_time = time.time()
        cur.execute(create_table_query)
        conn.commit()
        
        end_time = time.time()
        elapsed_time = end_time - start_time; 
        elapsed_time_2 = end_time - start_time_connection; 
        data_collection['create_user_item_table'] = elapsed_time_2
        # data_collection['create_user_item_table'].append(elapsed_time)
        
        print(f"Time taken to create users-items table and establishing connections : {elapsed_time_2:.6f} seconds")
        print(f"Time taken to create users-items table with established connections : {elapsed_time:.6f} seconds")
        
        print("Table 'users_items' created successfully")

        # Close the cursor and connection
        cur.close()
        conn.close()
        print("Connection closed")

    except Exception as e:
        print(f"Unable to connect to the database or create table: {e}")


# insert users

In [12]:
def measure_insert_one_user():
    start_time = time.time()
    try:
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        # Create a cursor object
        cur = conn.cursor()
        value = ("random_first_name", "random_last_name")
        insert_query = '''
        INSERT INTO users (first_name, last_name) VALUES (%s, %s);
        '''
        
        start_time_without_connection = time.time()
        cur.execute(insert_query, value)
        conn.commit()
        end_time = time.time()
        
        elapsed_time = end_time - start_time
        elapsed_time_without_connection = end_time - start_time_without_connection
        
        data_collection['insert_one_user_with_connection'] = elapsed_time
        data_collection['insert_one_user_without_connection'] = elapsed_time_without_connection

        print(f"insert one user with connectino:{elapsed_time:6f}")
        print(f"insert one user without connectino:{elapsed_time_without_connection:6f}")
       
        cur.close()
        conn.close()
    except Exception as e:
        print(f"Unable to connect to the database or insert values: {e}")

In [13]:
def insert_user(value):
    try:
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        # Create a cursor object
        cur = conn.cursor()
        insert_query = '''
        INSERT INTO users (first_name, last_name) VALUES (%s, %s);
        '''
        cur.execute(insert_query, value)
        conn.commit()
        cur.close()
        conn.close()
    except Exception as e:
        data_collection['insert_many_users_exception'] = data_collection['insert_many_users_exception'] + 1
        print(f"Unable to connect to the database or insert values: {e}")

def measure_insert_users(num_users):
    values = [(f"first_name_{i}", f"last_name_{i}") for i in range(num_users)]


    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(insert_user, values)
    end_time = time.time()

    elapsed_time = end_time - start_time

    data_collection[f'insert_{num_users}_users'] = elapsed_time

    print(f"Time taken to insert 1000 users: {elapsed_time:.6f} seconds")


# measure user exception


In [14]:
# def measure_users_exceptions(num_users): 
#     '''
#     measure the number of exceptoins of inserting 10000 users
#     '''
#     drop_all_tables()
#     measure_creating_users_table()
#     measure_insert_users(10000)
#     measure_create_items_table()
#     measure_create_users_items_table() 
#     measure_insert_users(num_users)

# insert items


In [15]:
def insert_item(value):
    try:
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        # Create a cursor object
        cur = conn.cursor()
        insert_query = '''
        INSERT INTO items (item_name,value1) VALUES (%s,%s);
        '''
        cur.execute(insert_query, value)
        conn.commit()
        cur.close()
        conn.close()
    except Exception as e:
        data_collection['insert_many_items_exception'] = data_collection['insert_many_items_exception'] + 1
        print(f"Unable to connect to the database or insert values: {e}")

def measure_insert_items(num_items):
    item_values = [(f"item_name_{i}",'ramdon_value') for i in range(num_items)]

    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(insert_item, item_values)
    end_time = time.time()

    elapsed_time = end_time - start_time

    data_collection[f'insert_{num_items}_items'] = elapsed_time

    print(f"Time taken to insert 1000 items: {elapsed_time:.6f} seconds")


# insert users-items

In [16]:
def get_userid_itemid(): 
    try:
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        print("get_userid_itemid(): Connected to the database successfully")

        # Create a cursor object
        cur = conn.cursor()

        # Retrieve all user IDs
        cur.execute("SELECT user_id FROM users;")
        user_ids = [row[0] for row in cur.fetchall()]

        # Retrieve all item IDs
        cur.execute("SELECT item_id FROM items;")
        item_ids = [row[0] for row in cur.fetchall()]

        # Close the cursor and connection
        cur.close()
        conn.close()
        print("get_userid_itemid(): Connection closed")
        return user_ids, item_ids
    
    except Exception as e:
        print(f"Unable to connect to the database or retrieve IDs: {e}")

In [17]:
def measure_one_insert_user_items_table():
    try:
        user_ids, item_ids = get_userid_itemid()
        # Insert relationships into 'users_items' table
        insert_query = '''
        INSERT INTO users_items (user_id, item_id) VALUES (%s, %s);
        '''
        
        start_time_connection = time.time() 
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        print("Connected to the  database successfully")

        # Create a cursor object
        cur = conn.cursor()
        
        start_time_without_connection = time.time()
        user_id = user_ids[0]
        
        # Randomly select 5 items for user_id
        random_items = random.sample(item_ids, 5)
        for item_id in random_items:
            cur.execute(insert_query, (user_id, item_id))

        conn.commit()
        end_time = time.time()
        
        elapsed_time_wihtout_connection = end_time - start_time_without_connection
        elapsed_time_with_connection = end_time - start_time_connection
        
        data_collection['insert_5_items_one_users_with_connection'] = elapsed_time_with_connection
        data_collection['insert_5_items_one_users_without_connection'] = elapsed_time_wihtout_connection
        
        print(f"insert user-items table with connections : {elapsed_time_with_connection:.6f} seconds")
        print(f"insert user-items table without connections : {elapsed_time_wihtout_connection:.6f} seconds")
        
        print("Values inserted successfully into items table")

        print("Relationships inserted successfully into 'users_items' table")

        # Close the cursor and connection
        cur.close()
        conn.close()
        print("Connection closed")

    except Exception as e:
        print(f"Unable to connect to the database or insert relationships: {e}")

In [18]:
def insert_user_items(user_id, item_ids):
    try:
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        # Create a cursor object
        cur = conn.cursor()
        insert_query = '''
        INSERT INTO users_items (user_id, item_id) VALUES (%s, %s);
        '''
        # Randomly select 5 items for each user
        random_items = random.sample(item_ids, 5)
        for item_id in random_items:
            cur.execute(insert_query, (user_id, item_id))

        conn.commit()
        cur.close()
        conn.close()
    except Exception as e:
        data_collection['insert_5_items_per_users_exception'] = data_collection['insert_5_items_per_users_exception'] + 1
        print(f"Unable to connect to the database or insert relationships: {e}")

def measure_insert_user_items_table():
    user_ids, item_ids = get_userid_itemid()

    start_time = time.time()
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        executor.map(insert_user_items, user_ids, [item_ids]*len(user_ids))
    end_time = time.time()

    elapsed_time = end_time - start_time

    data_collection['insert_5_items_per_users'] = elapsed_time

    print(f"Time taken to insert 5 items per user with established connections: {elapsed_time:.6f} seconds")


# query with given firstname and lastname

In [19]:
def measure_one_query():
    first_name = 'first_name_2'
    last_name = 'last_name_2'
    try:
        # Define the SQL query
        query = '''
        SELECT i.item_name
        FROM users u
        JOIN users_items ui ON u.user_id = ui.user_id
        JOIN items i ON ui.item_id = i.item_id
        WHERE u.first_name = %s AND u.last_name = %s;
        '''
        start_time_connection = time.time()
        
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        # Create a cursor object
        cur = conn.cursor()
        
        # Execute the SQL query
        start_time = time.time()
        cur.execute(query, (first_name, last_name))
        
        # Fetch and return the results
        items = cur.fetchall()
        end_time = time.time()
        
        elapsed_time_with_connection = end_time - start_time_connection 
        elapsed_time = end_time - start_time 
        
        print(f"query one user with connection {elapsed_time_with_connection} seconds")
        print(f"query one user without connection {elapsed_time} seconds")
        
        data_collection['query_one_with_connection'] = elapsed_time_with_connection 
        data_collection['query_one_without_connection'] = elapsed_time
        
        # Close the cursor and connection
        cur.close()
        conn.close()
        
        return items

    except Exception as e:
        data_collection['query_users_exception'] = data_collection['query_users_exception'] + 1
        print(f"Unable to connect to the database or execute query: {e}")
        return []


In [20]:
def execute_query(first_name, last_name):
    try:
        # Define the SQL query
        query = '''
        SELECT i.item_name
        FROM users u
        JOIN users_items ui ON u.user_id = ui.user_id
        JOIN items i ON ui.item_id = i.item_id
        WHERE u.first_name = %s AND u.last_name = %s;
        '''
        # Connect to the PostgreSQL server and 'users' database
        conn = psycopg2.connect(
            host=host,
            port=port,
            dbname=dbname,
            user=user,
            password=password
        )
        # Create a cursor object
        cur = conn.cursor()
        
        # Execute the SQL query
        cur.execute(query, (first_name, last_name))
        
        # Fetch and return the results
        items = cur.fetchall()
        
        # Close the cursor and connection
        cur.close()
        conn.close()
        
        return items

    except Exception as e:
        data_collection['query_users_exception'] = data_collection['query_users_exception'] + 1
        print(f"Unable to connect to the database or execute query: {e}")
        return []

def measure_query(num_queries):
    # Define the users for the query
    users = [(f"first_name_{i}", f"last_name_{i}") for i in range(num_queries)]

    start_time = time.time()
    results = []
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        future_to_user = {executor.submit(execute_query, first_name, last_name): (first_name, last_name) for first_name, last_name in users}
        for future in concurrent.futures.as_completed(future_to_user):
            user = future_to_user[future]
            try:
                data = future.result()
                results.extend(data)
            except Exception as e:
                print(f"Query failed for user {user}: {e}")

    end_time = time.time()
    elapsed_time = end_time - start_time
    data_collection['query_users'] = elapsed_time

    print(f"Time taken to query {num_queries:d} users from table with three joins: {elapsed_time:.6f} seconds")

    # Print the results
    print(len(results))
    # for item in results:
    #     print(item[0])

# measure time function calls

In [21]:
# drop_all_tables()

In [22]:
# measure_connection_time()

In [23]:
# measure_creating_users_table()

In [24]:
# measure_create_items_table()

In [25]:
# measure_create_users_items_table()

In [26]:
# measure_insert_one_user()

In [27]:
# measure_insert_users(num_users = num_users)

In [28]:
# measure_users_exceptions(num_users=num_users)

In [29]:
# measure_insert_items(num_items=num_items)

In [30]:
# measure_one_insert_user_items_table()

In [31]:
# pprint(data_collection)

In [32]:
# measure_insert_user_items_table()

In [33]:
# measure_query(num_queries=num_queries)

In [34]:
# pprint(data_collection)

In [35]:
# try:
#     with open(file_name, 'w') as file:
#         json.dump(data_collection, file)
#     print(f"Data collection successfully saved to {file_name}")
# except Exception as e:
#     print(f"Error saving data collection to file: {e}")

In [36]:
# measure_one_query()

# concurrency measurement

In [37]:
for max_workers in [ 8, 16, 32]: 
    data_collection = defaultdict(int)
    file_name = f"t2.2xlarge_{max_workers}.data.json"
    drop_all_tables()
    measure_connection_time()
    measure_creating_users_table()
    measure_create_items_table()
    measure_create_users_items_table()
    measure_insert_one_user()
    measure_insert_users(num_users = num_users)
    measure_insert_items(num_items=num_items)
    measure_one_insert_user_items_table()
    measure_insert_user_items_table()
    measure_one_query()
    measure_query(num_queries=num_queries)
    
    try:
        with open(file_name, 'w') as file:
            json.dump(data_collection, file)
        print(f"Data collection successfully saved to {file_name}")
    except Exception as e:
        print(f"Error saving data collection to file: {e}")

Connected to the database successfully
Tables dropped successfully
Connection closed
Time taken to connect to the database: 0.286729 seconds
Time taken to fetch the version data : 0.080083 seconds
PostgreSQL version: ('PostgreSQL 16.3 (Ubuntu 16.3-0ubuntu0.24.04.1) on x86_64-pc-linux-gnu, compiled by gcc (Ubuntu 13.2.0-23ubuntu4) 13.2.0, 64-bit',)
Connection closed
Time taken to create users table and establishing connections : 0.411290 seconds
Time taken to create users table with established connections : 0.125825 seconds
Table 'users' created successfully
Connection closed
Connected to the database successfully
Time taken to create items table and establishing connections : 0.427026 seconds
Time taken to create items table with established connections : 0.126764 seconds
Table 'items' created successfully
Connection closed
Connected to the database successfully
Time taken to create users-items table and establishing connections : 0.418139 seconds
Time taken to create users-items tabl

t2.micro_8 
20% cpu inserting 1000 users, 
20% cpu utilization 1000 items, 
18% cpu inserting about 5000 users-items; 
25% cpu query;

t2.micro_16 
35% cpu inserting 1000 users, 
35% cpu utilization 1000 items, 
31% cpu inserting about 5000 users-items; 
50% cpu query;

t2.micro_32 
70% cpu inserting 1000 users, 
70% cpu utilization 1000 items, 
63% cpu inserting about 5000 users-items; 
80% cpu query;

t2.medium_8 
10% cpu x2 inserting 1000 users, 
12% cpu x2 utilization 1000 items, 
10% cpu inserting about 5000 users-items; 
25% cpu query;

t2.medium_16
60% cpu x2 inserting 1000 users, 
59% cpu x2 utilization 1000 items, 
80%, 52% cpu inserting about 5000 users-items; 
70%, 60% cpu query;

t2.medium_32
80% cpu x2 inserting 1000 users, 
59% cpu x2 utilization 1000 items, 
80%, 52% cpu inserting about 5000 users-items; 
30%, 60% cpu query;

t2.xlarge_8
5% cpu x2 inserting 1000 users, one thread always run faster than other threads
5% cpu x2 utilization 1000 items, 
5% cpu inserting about 5000 users-items; 
9%, other 6% cpu query;

t2.xlarge_16
10%,other 8 inserting 1000 users, one thread always run faster than other threads
10% cpu x4 utilization 1000 items, 
5% cpu inserting about 5000 users-items; 
9%, other 6% cpu query;